In [2]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import SVD, SVDpp, NMF, BaselineOnly, NormalPredictor

%matplotlib inline

In [8]:
#Import our meta data 
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta_data_df = getDF('Data/meta_Beauty.json.gz')

In [10]:
meta_data_df.head(3)

,asin,description,title,imUrl,salesRank,categories,price,related,brand
0,0205616461,"As we age, our once youthful, healthy skin suc...",Bio-Active Anti-Aging Serum (Firming Ultra-Hyd...,http://ecx.images-amazon.com/images/I/41DecrGO...,{'Health & Personal Care': 461765},"[[Beauty, Skin Care, Face, Creams & Moisturize...",NaN,NaN,NaN
1,0558925278,Mineral Powder Brush--Apply powder or mineral ...,Eco Friendly Ecotools Quality Natural Bamboo C...,http://ecx.images-amazon.com/images/I/51L%2BzY...,{'Beauty': 402875},"[[Beauty, Tools & Accessories, Makeup Brushes ...",NaN,NaN,NaN
2,0733001998,"From the Greek island of Chios, this Mastiha b...",Mastiha Body Lotion,http://ecx.images-amazon.com/images/I/311WK5y1...,{'Beauty': 540255},"[[Beauty, Skin Care, Body, Moisturizers, Lotio...",NaN,NaN,NaN


In [11]:
reviews_df = pd.read_json("Data/reviews_Beauty_5.json.gz",lines=True)

In [18]:
unique_asins_in_reviews = list(reviews_df.asin.unique())
len(unique_asins_in_reviews)

12101

In [20]:
meta_data_df_cleaned = meta_data_df[meta_data_df.asin.isin(unique_asins_in_reviews)]
meta_data_df_cleaned

,asin,description,title,imUrl,salesRank,categories,price,related,brand
115,7806397051,An extensive range of 15 multiple vibrant long...,WAWO 15 Color Professionl Makeup Eyeshadow Cam...,http://ecx.images-amazon.com/images/I/41Rn18Oe...,{'Beauty': 10486},"[[Beauty, Makeup, Face, Concealers & Neutraliz...",5.04,"{'also_bought': ['B00KR26VFE', 'B00E7LQHZ0', '...",COKA
179,9759091062,Xtreme Brite Brightening gel is a highly conc...,Xtreme Brite Brightening Gel 1oz.,http://ecx.images-amazon.com/images/I/41QWW9v1...,{'Beauty': 52254},"[[Beauty, Hair Care, Styling Products, Creams,...",19.99,"{'also_bought': ['B0054GLD1U', 'B003BRZCUC', '...",Xtreme Brite
192,9788072216,Prada Candy By Prada Eau De Parfum Spray 1.7 O...,Prada Candy By Prada Eau De Parfum Spray 1.7 O...,http://ecx.images-amazon.com/images/I/51iT2k6L...,{'Beauty': 78916},"[[Beauty, Fragrance, Women's, Eau de Parfum]]",65.86,"{'also_bought': ['B006C5OHSI', 'B006P14842', '...",Prada
555,9790790961,Versace Bright Crystal Perfume for Women 3 oz ...,Versace Bright Crystal Eau de Toilette Spray f...,http://ecx.images-amazon.com/images/I/418LYGLE...,{'Beauty': 764},"[[Beauty, Fragrance, Women's, Eau de Toilette]]",52.33,"{'also_bought': ['B007P7OPQQ', 'B0017JT658', '...",Versace
587,9790794231,STELLA For Women By STELLA MCCARTNEY 1.7 oz ED...,Stella McCartney Stella,http://ecx.images-amazon.com/images/I/31L2n60J...,{'Beauty': 142503},"[[Beauty, Fragrance, Women's, Eau de Parfum]]",NaN,"{'also_bought': ['B0019M21OQ', 'B000E7YM8K', '...",NaN
...,...,...,...,...,...,...,...,...,...
259140,B00L5JHZJO,We Only Source Our 100% Organic & Pure Argan O...,"Moroccan Argan Oil - For Hair, Face, Skin, &am...",http://ecx.images-amazon.com/images/I/41kTNm0k...,{'Beauty': 120},"[[Beauty, Skin Care, Body, Moisturizers, Oils]]",14.99,"{'also_bought': ['B00JSLU7MS', 'B00GZ0WFQG', '...",Natural Beauty
259143,B00L6Q3BH6,Lip stain that goes on liquid and dries comple...,LIME CRIME Velvetines - Wicked,http://ecx.images-amazon.com/images/I/41q7jpgt...,{'Health & Personal Care': 3006},"[[Beauty, Makeup, Lips, Lipstick]]",27.50,"{'also_bought': ['B00JH29K6M', 'B00JH2B182', '...",Lime Crime
259170,B00LCEROA2,NaN,Dr Song Rosehip Oil 4oz (4 oz),http://ecx.images-amazon.com/images/I/412qdoPc...,{'Beauty': 7597},"[[Beauty, Skin Care, Face, Oils & Serums]]",19.99,"{'also_bought': ['B00LNVW1IE', 'B00JYKGFWY', '...",NaN
259179,B00LG63DOM,The Secret to Younger Looking Skin Vitamin C S...,VITAMIN C SERUM 20% with Hyaluronic Acid For Y...,http://ecx.images-amazon.com/images/I/31JTTyCU...,{'Beauty': 58521},"[[Beauty, Skin Care, Face, Creams & Moisturize...",36.00,"{'also_bought': ['B00JJOEV9Y', 'B00KWP8ZIM', '...",NaN
